# Azure AI Search LangChain vector code sample
This code demonstrates how to use Azure AI Search with OpenAI and Langchain

### Set up a Python virtual environment in Visual Studio Code

1. Open the Command Palette (Ctrl+Shift+P).
1. Search for **Python: Create Environment**.
1. Select **Venv**.
1. Select a Python interpreter. Choose 3.10 or later.

It can take a minute to set up. If you run into problems, see [Python environments in VS Code](https://code.visualstudio.com/docs/python/environments).

### Install packages

In [1]:
! pip install -r azure-search-vector-python-langchain-sample-requirements.txt --quiet


[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


### Load .env file (Copy .env-sample to .env and update accordingly)

In [2]:
from dotenv import load_dotenv
from azure.identity import DefaultAzureCredential
import os

load_dotenv(override=True) # take environment variables from .env.

# Variables not used here do not need to be updated in your .env file
endpoint = os.environ["AZURE_SEARCH_SERVICE_ENDPOINT"]
key_credential = os.environ["AZURE_SEARCH_ADMIN_KEY"] if len(os.environ["AZURE_SEARCH_ADMIN_KEY"]) > 0 else None
index_name = os.environ["AZURE_SEARCH_INDEX"]
azure_openai_endpoint = os.environ["AZURE_OPENAI_ENDPOINT"]
azure_openai_key = os.environ["AZURE_OPENAI_KEY"] if len(os.environ["AZURE_OPENAI_KEY"]) > 0 else None
azure_openai_embedding_deployment = os.environ["AZURE_OPENAI_EMBEDDING_DEPLOYMENT"]
azure_openai_api_version = os.environ["AZURE_OPENAI_API_VERSION"]

credential = key_credential or DefaultAzureCredential()

### Create LangChain Azure OpenAI Embeddings

In [5]:
from langchain_openai import AzureOpenAIEmbeddings
from azure.identity import DefaultAzureCredential, get_bearer_token_provider

openai_credential = DefaultAzureCredential()
token_provider = get_bearer_token_provider(openai_credential, "https://cognitiveservices.azure.com/.default")

# Use API key if provided, otherwise use RBAC authentication
embeddings = AzureOpenAIEmbeddings(
    azure_deployment=azure_openai_embedding_deployment,
    openai_api_version=azure_openai_api_version,
    azure_endpoint=azure_openai_endpoint,
    api_key=azure_openai_key,
    azure_ad_token_provider=token_provider if not azure_openai_key else None
)   

### Create LangChain Vector Store

In [6]:
from langchain.vectorstores.azuresearch import AzureSearch

vector_store = AzureSearch(
    azure_search_endpoint=endpoint,
    azure_search_key=key_credential,
    index_name=index_name,
    embedding_function=embeddings.embed_query,
    semantic_configuration_name="default"
)

In [7]:
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
import os

directory = os.path.join("..", "data", "documents")
files = ["Benefit_Options.pdf", "Northwind_Health_Plus_Benefits_Details.pdf", "Northwind_Standard_Benefits_Details.pdf"]
total_chunks = 0
splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=0)

for file in files:
    loader = PyPDFLoader(os.path.join(directory, file))
    file_chunks = loader.load_and_split(splitter)
    results = vector_store.add_documents(documents=file_chunks)
    total_chunks += len(results)
    print(f"Indexed {file}")
print(f"Indexed {total_chunks} chunks")

Indexed Benefit_Options.pdf
Indexed Northwind_Health_Plus_Benefits_Details.pdf
Indexed Northwind_Standard_Benefits_Details.pdf
Indexed 636 chunks


## Perform a vector similarity search

In [8]:
# Perform a similarity search
docs = vector_store.similarity_search(
    "What is included in my Northwind Health Plus plan that is not in standard?",
    k=3,
    search_type="similarity",
)
for doc in docs:
    print("-" * 80)  
    print(f"Source: {doc.metadata['source']}")
    print(f"Chunk Content: {doc.page_content}")

--------------------------------------------------------------------------------
Source: ..\data\documents\Northwind_Health_Plus_Benefits_Details.pdf
Chunk Content: Specialty Care Services: Northwind  Health Plus covers specialty care services, such as 
physical therapy, occupational therapy, and mental health services. The plan also covers 
services provided by specialists, such as cardiologists, endocrinologists, and neurologists.  
Exceptions: Northwin d Health Plus does not cover services that are not medically 
necessary, such as cosmetic surgery, elective treatments, and experimental treatments. In 
addition, the plan does not cover services for conditions that are not covered by the plan, 
such as pre -existing conditions.
--------------------------------------------------------------------------------
Source: ..\data\documents\Northwind_Health_Plus_Benefits_Details.pdf
Chunk Content: is important to understand these exclusions so that you can plan your care accordingly.  
Servi

## Perform a hybrid search

In [9]:
# Perform a hybrid search
docs = vector_store.similarity_search(
    query="What is included in my Northwind Health Plus plan that is not in standard?",
    k=3, 
    search_type="hybrid"
)
for doc in docs:
    print("-" * 80)  
    print(f"Source: {doc.metadata['source']}")
    print(f"Chunk Content: {doc.page_content}")

--------------------------------------------------------------------------------
Source: ..\data\documents\Northwind_Health_Plus_Benefits_Details.pdf
Chunk Content: is important to understand these exclusions so that you can plan your care accordingly.  
Services Not Covered:  
Northwind Health Plus does not cover services that are not medically necessary, such as 
cosmetic surgery or elective procedures. Additionally, services or treatments that are 
experimental or investigational are not covered under this plan.  
Prescriptions N ot Covered: The plan does not cover prescriptions that are not medically 
necessary, certain over -the-counter medications, or prescription medications that are used 
to enhance performance in athletics.  
Mental Health and Substance Abuse Treatment: The plan does not cover mental health or 
substance abuse treatment services provided by a non -network provider or any services 
that are not medically necessary.  
Preventive Care: Northwind Health Plus does 

## Perform a hybrid search with semantic reranking (powered by Bing)

In [10]:
# Perform a hybrid search with semantic reranking  
docs_and_scores = vector_store.semantic_hybrid_search_with_score(  
    query="What is included in my Northwind Health Plus plan that is not in standard?",  
    k=3,  
)  
  
# Print the results  
for doc, score in docs_and_scores:  
    print("-" * 80)  
    answers = doc.metadata['answers']  
    if answers:  
        if answers.get('highlights'):  
            print(f"Semantic Answer: {answers['highlights']}")  
        else:  
            print(f"Semantic Answer: {answers['text']}")  
        print(f"Semantic Answer Score: {score}")  
    print("Content:", doc.page_content)  
    captions = doc.metadata['captions']
    print(f"Score: {score}") 
    if captions:  
        if captions.get('highlights'):  
            print(f"Caption: {captions['highlights']}")  
        else:  
            print(f"Caption: {captions['text']}")  
    else:  
        print("Caption not available")  


--------------------------------------------------------------------------------
Content: is important to understand these exclusions so that you can plan your care accordingly.  
Services Not Covered:  
Northwind Health Plus does not cover services that are not medically necessary, such as 
cosmetic surgery or elective procedures. Additionally, services or treatments that are 
experimental or investigational are not covered under this plan.  
Prescriptions N ot Covered: The plan does not cover prescriptions that are not medically 
necessary, certain over -the-counter medications, or prescription medications that are used 
to enhance performance in athletics.  
Mental Health and Substance Abuse Treatment: The plan does not cover mental health or 
substance abuse treatment services provided by a non -network provider or any services 
that are not medically necessary.  
Preventive Care: Northwind Health Plus does not cover preventive care services provided 
by a non -network  provider.  